# Week 1: Introduction to Computer Vision

## Notebook 3: Semantic Segmentation with a U-Net Convolutional Neural Network using PyTorch

Welcome to the fourth notebook of this week's Applied AI Study Group! We will study semantic segmentation problem with MSRC-v2 image dataset provided by Microsoft. The aim of our task will be to make object segmentation in the given images.

### 1. Semantic Segmentation

Semantic Segmentation aims to label each pixel (aka classify pixel-wise) of a given image. We treat different objects of the same class as they are same object. On contrast, instance segmentation treats each objects of the same class as they are different objects, hence, label them differently such as object 1, object 2, etc. In this notebook, we will tackle the problem of semantic segmentation. The pixel-wise operations can be applied via segmentation on images, for example, portrait mode in images requires to differentiate between foreground and background of an image. We blur out the pixels which are classified as background. 

So, how do we build our model for this case? We know that the capabilities of convolution filters are proven in terms of their capabilities in processing structured data such as images. However, they reduce the size of their input vector depends on their kernel size. We need a model that outputs the same size of input vector since we want to retrieve the same image we give into the model. Luckily for us, we have U-Net Architecture for these kind of tasks. We will study U-Nets in the following section.

### 2. U-Net Convolutional Neural Network

![U-Net Model Architecture](./images/u-net.jpg "Example U-Net")

We can divide the U-Net architecture into two parts while studying it: the left half of the network is responsible for information encoding and the right half of the network is responsible for information decoding.

* Encoder captures the context of image with a series of convolution and pooling operations, hence, it is responsible for feature extraction from the input image. 
* The middle layer between the encoder and the decoder is called bottlenecek representation of the input image. It is the high level feature representation of input data, e.g. objects and events. Using the information retrieved from each layer, we re-construct an output image with a same size as input image. For that, we utilize decoder of U-Net.
* Decoder upsamples the bottleneck representation to recover spatial locations (aka pixel-wise information) for assigning class labels to each pixel of the input image. For that, we recover the same size output image as the input image. During upsampling, we add extracted features from encoder part as in skip connections. We cannot afford to lose much information during encoding-decoding. Therefore, we need low-level features such as edges to have better results in classifying pixels.

### 3. Imports and Checks

You should have installed Numpy and Matplotlib using `pip` and, PyTorch using [Week 0 - Notebook 2](https://github.com/inzva/Applied-AI-Study-Group/blob/add-frameworks-week/Applied%20AI%20Study%20Group%20%236%20-%20January%202022/Week%200/2-mnist_classification_convnet_pytorch.ipynb).

The python file segmentation_dataset.py and MSRC-v2 image dataset can be found in the following link: [Segmentation](https://drive.google.com/drive/u/1/folders/18bKQKwvjuXbjNDMI-ktRD6tet9tcxNkL)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import torch
from datasets.segmentation_dataset import SegmentationData, label_img_to_rgb

If the following two code cells are running successfully, then, you are good to go further!

In [ ]:
print(torch.__version__)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

We will load the data using the code provided in segmentation_dataset.py

In [ ]:
data_root = os.path.join('./datasets','segmentation')

train_data = SegmentationData(image_paths_file=f'{data_root}/segmentation_data/train.txt')
val_data = SegmentationData(image_paths_file=f'{data_root}/segmentation_data/val.txt')
test_data = SegmentationData(image_paths_file=f'{data_root}/segmentation_data/test.txt')

The first print calls are for double-checking the data loading.

Then, we visualize a couple of example images to observe what our input images look like and what our output images should look like.

In [ ]:
print("Train size: %i" % len(train_data))
print("Validation size: %i" % len(val_data))
print("Img size: ", train_data[0][0].size())
print("Segmentation size: ", train_data[0][1].size())

num_example_imgs = 4
plt.figure(figsize=(10, 5 * num_example_imgs))
for i, (img, target) in enumerate(train_data[:num_example_imgs]):
    # img
    plt.subplot(num_example_imgs, 2, i * 2 + 1)
    plt.imshow(img.numpy().transpose(1,2,0))
    plt.axis('off')
    if i == 0:
        plt.title("Input image")
    
    # target
    plt.subplot(num_example_imgs, 2, i * 2 + 2)
    plt.imshow(label_img_to_rgb(target.numpy()))
    plt.axis('off')
    if i == 0:
        plt.title("Target image")
plt.show()

We will build our model in the following cell. Since semantic segmentation is a challenging task, we will use a pretrained model from torchvision for the encoder of our U-Net architecture. Then, we will build our decoder using Upsample, ConvTranspose2d, and LeakyReLU activation layers on top of MobileNet encoder.

One important note for pretrained model: We use [MobileNet](https://arxiv.org/pdf/1704.04861.pdf) for its fast inference time. We don't need a more complex model with much inference time for the purpose of this notebook. You can check the models provided by [torchvision](https://pytorch.org/vision/0.8/models.html) for trying out different models. The MobileNet is trained with [ImageNet dataset](https://www.image-net.org/index.php). It is trained for classification task.  Hence, you can see that we exclude the last layer of MobileNet because we don't need the classifier layer but we need the rest of the network for feature extraction.

In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models

class SegmentationNN(nn.Module):

    def __init__(self, num_classes=23, hparams=None):
        super().__init__()
        self.hparams = hparams

        mobile_network = models.mobilenet_v2(pretrained=True)

        layers = list(mobile_network.children())[:-1]  # 1x1280x8x8
        layers.append(nn.Conv2d(1280, 120, 1, 1))  # 1x160x8x8
        layers.append(nn.LeakyReLU(0.1))
        layers.append(nn.Upsample(scale_factor=4))  # 1x160x32x32
        layers.append(nn.ConvTranspose2d(120, 80, 3, 2))  # 1x120x64x64
        layers.append(nn.LeakyReLU(0.1))
        layers.append(nn.ConvTranspose2d(80, 60, 9, dilation=2))  # 1x80x80x80
        layers.append(nn.LeakyReLU(0.1))
        layers.append(nn.ConvTranspose2d(60, 40, 9, dilation=2))  # 1x60x96x96
        layers.append(nn.LeakyReLU(0.1))
        layers.append(nn.ConvTranspose2d(40, 40, 11, dilation=2))  # 1x40x116x116
        layers.append(nn.LeakyReLU(0.1))
        layers.append(nn.Upsample(scale_factor=2))  # 1x40x232x232
        layers.append(nn.ConvTranspose2d(40, 23, 7))  # 1x23x240x240
        layers.append(nn.LeakyReLU(0.1))
        self.network = nn.Sequential(*layers)

    def forward(self, x):

        x = self.network(x)

        return x

We specify our training hyperparameters and set the rest of the adjustments for training.

In [ ]:
hparams = {
    "lr" : 0.001,
    "batch_size" : 4,
    "num_epochs" : 4
}  

model = SegmentationNN(hparams=hparams)

model.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=hparams["lr"])
criterion = torch.nn.CrossEntropyLoss(ignore_index=-1, reduction='mean')

train_loader = torch.utils.data.DataLoader(train_data, batch_size=hparams["batch_size"], shuffle=True)
print(train_loader)
print(next(iter(train_loader)))

We do a test run for training loop to check if anything is wrong.

In [ ]:
for (inputs, targets) in train_data[0:4]:
    inputs, targets = inputs, targets
    outputs = model(inputs.unsqueeze(0).to(device))
    losses = criterion(outputs, targets.unsqueeze(0).to(device))
    print(losses)

Now, we train our model using the classical PyTorch training loop.

In [ ]:
print('training starts!')
for epoch in range(hparams["num_epochs"]):
    
    epoch_loss = 0.0
    
    for i, data in enumerate(train_loader):
        images, labels = data[0].to(device), data[1].to(device)
        optimizer.zero_grad()
        predictions = model(images)
        loss = criterion(predictions, labels)
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
    print("Epoch: %d Loss: %.3f" % (epoch + 1, epoch_loss / 276))

In [ ]:
#TODO: add validation into training loop for per epoch
#TODO: add test code and visualize some results.